

# MultiModel
This notebook shows the movie example from my talk/blog post on using Amazon Neptune to help model a multi-model database solution.

In this notebook you insert into Neptune two types of data:
- An ontology of data products.
- Specific movie resource data. This shows how the Knowledge Graph model within the overall metamodel works. 

## Add Ontology to Neptune


### Set the name of an S3 bucket in the same region that Neptune has access to
If necessary, add IAM role to Neptune cluster allowing it read access.

In [ ]:
S3_BUCKET='s3://your-bucket-name/your-folder' 


### Download the TTL files containing the MM ontology. Then move to S3

In [ ]:
%%bash -s "$S3_BUCKET"

mkdir ttl
cd ttl
rm *
wget https://raw.githubusercontent.com/aws-samples/amazon-neptune-ontology-example-blog/main/multimodel/mm_movie.ttl
aws s3 sync . $1


### Bulk-load to Neptune from S3

In [ ]:
%load -s {S3_BUCKET} -f turtle --store-to loadres --run

### Check status of load

In [ ]:
%load_status {loadres['payload']['loadId']} --errors --details

## Query the products (and some KG instances)

### Get list of products

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

select ?product where {
    ?product rdfs:subClassOf+ :DataProduct .
} 
ORDER BY ?product 

### Get list of products and their impls

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

select ?product (GROUP_CONCAT(?impl;SEPARATOR=",") AS ?impls) where {
    ?product rdfs:subClassOf+ :DataProduct .
    OPTIONAL { ?product :hasImpl ?impl } .
} 
GROUP BY ?product
ORDER BY ?product 

### Inspect one product a few levels deep

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

select * where {
    ?anno a owl:AnnotationProperty .
    movdoc:MovieDocument ?anno ?o .
    OPTIONAL { 
        ?anno2 a owl:AnnotationProperty  .
        ?o ?anno2 ?o2 . 
    } .
} 
ORDER BY ?anno ?o ?anno2 

### Describe a product

In [ ]:
%%sparql

# describe mode https://docs.aws.amazon.com/neptune/latest/userguide/sparql-query-hints-for-describe.html#sparql-query-hints-describeMode

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 
PREFIX hint: <http://aws.amazon.com/neptune/vocab/v01/QueryHints#>

describe movkg:MovieResource
#{
#  hint:Query hint:describeMode "CBD"
#}

### Which products use OpenSearch and Elasticache?

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

select * where {
    ?product rdfs:subClassOf :DataProduct .
    ?product :hasImpl/:hasService aws:OpenSearch .
    ?product :hasImpl/:hasService aws:Elasticache .
}

### Story and Movie Related?

In [ ]:
%%sparql
PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

ask where {
    BIND(movstory:StoryAnalysis as ?product) .
    
    ?product  ((:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf) |^ 
        (:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf))* ?rel .

    FILTER(?rel = movkg:MovieResource || ?rel = movdoc:MovieDocument) . 
} 


### Story and IMDB?

In [ ]:
%%sparql
PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

ask where {
    BIND(movstory:StoryAnalysis as ?product) .
    
    ?product  ((:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf) |^ 
        (:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf))* ?rel .

    FILTER(?rel = mov:IMDB) . 
} 

### Story and the Lonely product

In [ ]:
%%sparql
PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

ask where {
    BIND(movstory:StoryAnalysis as ?product) .
    
    ?product  ((:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf) |^ 
        (:hasNeighbor|:hasNeighborAttribute|:joins|:refersTo|:hasSimilarity|
        :hasSource|:hasDataset|rdfs:subPropertyOf|owl:hasKey/rdf:first|rdfs:domain|rdfs:range|rdfs:subPropertyOf))* ?rel .

    FILTER(?rel = mov:LonelyProduct) . 
} 

## Movie Example

### Populate sample data. A movie, a couple of its roles, stories that mention, video analysis, links to IMDB, DBPedia, Wikidata

In [ ]:
%%sparql
PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

INSERT DATA { graph <:mmblog> {
    movkg:Shining a movkg:MovieResource .
    movkg:Shining movdoc:MovieID "tt0081505" .
    movkg:Shining movkg:hasDBPediaRef <http://dbpedia.org/resource/The_Shining_(film)> .
    movkg:Shining movkg:hasWikidataRef <http://www.wikidata.org/entity/Q186341> .
    
    # cast - a couple contributors to give the idea
    movkg:RoleShining_Jack a movkg:RoleResource .
    movkg:RoleShining_Jack movkg:hasMovie movkg:Shining .
    movkg:RoleShining_Jack movkg:hasContribClass movkg:Actor .
    movkg:RoleShining_Jack movkg:hasContrib movkg:JackNicholson .

    movkg::RoleShining_Kubrick_Dir a movkg:RoleResource .
    movkg::RoleShining_Kubrick_Dir movkg:hasMovie movkg:Shining .
    movkg::RoleShining_Kubrick_Dir movkg:hasContribClass movkg:Director .
    movkg::RoleShining_Kubrick_Dir movkg:hasContrib movkg:StanleyKubrick .

    movkg:RoleShining_Kubrick_Prod a movkg:RoleResource .
    movkg:RoleShining_Kubrick_Prod movkg:hasMovie movkg:Shining .
    movkg:RoleShining_Kubrick_Prod movkg:hasContribClass movkg:Producer .
    movkg:RoleShining_Kubrick_Prob movkg:hasContrib movkg:StanleyKubrick .

    movkg:JackNicholson a movkg:ContributorResource . 
    movkg:JackNihcolson movkg:ContribID "nm0000197" .
    movkg:JackNicholson movkg:hasDBPediaRef <http://dbpedia.org/resource/Jack_Nicholson> .
    movkg:JackNicholson movkg:hasWikidataRef <https://www.wikidata.org/entitiy/Q39792> .

    movkg:StanleyKubrick a movkg:ContributorResource . 
    movkg:StanleyKubrick movkg:ContribID "nm0000040" .
    movkg:StanleyKubrick movkg:hasDBPediaRef <http://dbpedia.org/resource/Stanley_Kubrick> .
    movkg:StanleyKubrick movkg:hasWikidataRef <https://www.wikidata.org/entitiy/Q2001> .

    # stories that mention
    movkg:Story_Staycation_in_Hollywood a movkg:StorytResource .
    movkg:Story_Staycation_in_Hollywood movstory:StoryTitle "Staycation in Hollywood" .
    movkg:Story_Staycation_in_Hollywood movkg:mentions movkg:Shining .
    movkg:Story_Starve_Cabin_Fever_Until_Spring a movkg:StorytResource .
    movkg:Story_Starve_Cabin_Fever_Until_Spring movstory:StoryTitle "Starve Cabin Fever Until Spring" .
    movkg:Story_Starve_Cabin_Fever_Until_Spring movkg:mentions movkg:Shining .
    
    # video analysis
    movkg:Analysis_123456789 a movkg:VideoAnalysisResource .
    movkg:Shining movkg:hasVideoAnalysis movkg:Analysis_123456789 .
    movkg:Analysis_123456789 movvideo:VideoID "123456789" .
    movkg:Analysis_123456789 movvideo:S3IngestLocation "s3://va_abcderfg_123456789/ingest" .
    movkg:Analysis_123456789 movvideo:S3AnalysisLocation "s3://va_abcderfg_123456789/analysis" .
    movkg:Analysis_123456789 movvideo:MP4FileName "0081505_shining.mp4" .
    movkg:Analysis_123456789 movkg:hasRekognitionCeleb movkg:Analysis_123456789_celeb0 .
    movkg:Analysis_123456789_celeb0 movkg:celebName "Jeff Bezos" .
    movkg:Analysis_123456789_celeb0 movkg:hasWikidataRef <http://www.wikidata.org/entity/Q312556> .
    movkg:Analysis_123456789_celeb0 movdoc:ContribID "nm1757263" . # this is an IMDB ID
    # more detail on occurences of cebel in video in S3AnalysisLocation given above
 
}}

### With MovieID (IMDBID) as input, get basic details of the movie

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

SELECT ?movie ?dbp ?wiki ?storyMention ?video ?mp4
WHERE 
{
    ?movie movdoc:MovieID "tt0081505" .
    ?movie a movkg:MovieResource .
    OPTIONAL {?movie movkg:hasDBPediaRef ?dbp . } .
    OPTIONAL {?movie movkg:hasWikidataRef ?wiki . } .
    
    # bring in story mentions
    OPTIONAL {?storyMention movkg:mentions ?movie . } .
    
    # bring in video analyis
    OPTIONAL {?movie movkg:hasVideoAnalysis ?video . ?video movvideo:MP4FileName ?mp4 . } .
} 


### Knowing the movie URI, DESCRIBE it

See https://docs.aws.amazon.com/neptune/latest/userguide/sparql-query-hints-for-describe.html for more on DESCRIBE in Neptune. Try the Graph view too.

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

describe movkg:Shining
#{
#  hint:Query hint:describeMode "CBD"
#}

### Get video analysis - celebs

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

SELECT ?movie ?mp4 ?celebName ?celebWikdata ?celebIMDB ?roleX
WHERE 
{
    ?movie movdoc:MovieID "tt0081505" .
    ?movie a movkg:MovieResource .
    
    ?movie movkg:hasVideoAnalysis ?video . 
    ?video movvideo:MP4FileName ?mp4 .
    OPTIONAL {
        # bring in celebs in video analysis
        ?video movkg:hasRekognitionCeleb ?celeb .
        ?celeb movkg:celebName ?celebName .
        ?celeb movkg:hasWikidataRef ?celebWikdata .
        ?celeb movdoc:ContribID ?celebIMDB .
        OPTIONAL {
            # Is the celeb a contributor in the movie
            ?roleX movkg:hasContributor ?contribX .
            ?contribX a movkg:ContributorResource .
            ?roleX movkg:hasMovie ?movie .
            ?contribX movdoc:ContribID ?celebIMDB . 
        }
    } 
} 


### Pull in DBPedia

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

SELECT ?p ?o 
WHERE 
{
    ?movie movdoc:MovieID "tt0081505" .
    ?movie a movkg:MovieResource .
    ?movie movkg:hasDBPediaRef ?dbp .
    SERVICE <https://dbpedia.org/sparql> {
        ?dbp ?p ?o . 
    }
}


### Pull in Wikidata

In [ ]:
%%sparql

PREFIX : <http://amazon.com/aws/wwso/neptune/demo/multimodel/> 
prefix mov:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/> 
prefix movkg:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/kg/> 
prefix movvideo:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/video/> 
prefix movstory:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/story/> 
prefix movlake:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/lake/> 
prefix movdoc:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/movmm/doc/> 
prefix aws:           <http://amazon.com/aws/wwso/neptune/demo/multimodel/aws/> 

SELECT ?p ?o 
WHERE 
{
    ?movie movdoc:MovieID "tt0081505" .
    ?movie a movkg:MovieResource .
    ?movie movkg:hasWikidataRef ?wiki .
    SERVICE <https://query.wikidata.org/sparql> {
        ?wiki ?p ?o . 
    }    
}


## Cleanup

In [ ]:
%%sparql

delete {?s ?p ?o} where {?s ?p ?o}